```
Enter - wchodzimy do komórki (do trybu edycji)
Escape - wychodzimy z komórki (do trybu poleceń)

a - tworzy komórkę powyżej (above)
b - tworzy komórkę poniżej (below)
x (dd) - usuwa komórkę
z - cofa operację

Shift + Enter - uruchamia komórkę
Ctrl + Enter - uruchamia komórkę
Alt + Enter - uruchamia komórkę

m - zmiana trybu na markdown
y - zmiana trybu na kod
```

In [4]:
from psycopg2.extras import RealDictCursor
import psycopg2

In [5]:
conn = psycopg2.connect(
    dbname="postgres", user="postgres", password="postgres", host="localhost",
    cursor_factory=RealDictCursor)
cursor = conn.cursor()

In [7]:
cursor.execute("SELECT * FROM cars")
results = cursor.fetchall()

In [18]:
for column in ["price", "brand"]:
    print(results[0][column])

32900
Škoda


In [20]:
import psycopg

In [3]:
conn = psycopg.connect(
    dbname="postgres", user="postgres", password="postgres", host="localhost")
cursor = conn.cursor()

---

### CREATE

In [9]:
create_query = """CREATE TABLE tasks(
    id SERIAL,
    description TEXT,
    priority SMALLINT,
    is_completed BOOLEAN
)
"""

In [10]:
cursor.execute(create_query)
conn.commit()

In [11]:
create_query = """CREATE TABLE users(
    id SERIAL,
    username VARCHAR(16),
    password VARCHAR(36),
    is_admin BOOLEAN
)
"""

In [12]:
cursor.execute(create_query)
conn.commit()

### INSERT

In [15]:
insert_query = "INSERT INTO tasks(description, priority, is_completed) VALUES('opis zadania', 2, true)"

In [17]:
conn.rollback()

In [18]:
cursor.execute(insert_query)
conn.commit()

#### Returning (INSERT)

In [23]:
insert_query = f"""INSERT INTO tasks (description, priority, is_completed)
                   VALUES ('zrobić obiad', 3, false) RETURNING *"""

In [26]:
cursor.execute(insert_query)

record = cursor.fetchone()
record

(2, 'zrobić obiad', 3, False)

In [28]:
record[0]

2

In [29]:
conn.commit()

### `%s`

In [30]:
conn = psycopg.connect(
    dbname="postgres", user="postgres", password="postgres", host="localhost",
    cursor_factory=psycopg.ClientCursor
)
cursor = conn.cursor()

In [31]:
insert_query = """INSERT INTO tasks (description, priority, is_completed)
                  VALUES (%s, %s, %s)  RETURNING *"""

cursor.execute(insert_query, ('opis zadania', 3, True))
record = cursor.fetchone()
record

(3, 'opis zadania', 3, True)

In [32]:
cursor.mogrify(insert_query, ('opis zadania', 3, True))

"INSERT INTO tasks (description, priority, is_completed)\n                  VALUES ('opis zadania', 3, true)  RETURNING *"

In [33]:
conn.commit()

---

In [36]:
id_number = "3; DROP TABLE test;"

query = f"UPDATE tasks SET priority=4 WHERE id = {id_number}"
query

'UPDATE tasks SET priority=4 WHERE id = 3; DROP TABLE test;'

In [37]:
cursor.execute(query)
conn.commit()

In [38]:
id_number = "3; DROP TABLE test;"

query = "UPDATE tasks SET priority=4 WHERE id = %s"
query

'UPDATE tasks SET priority=4 WHERE id = %s'

In [40]:
cursor.mogrify(query, (id_number,))

"UPDATE tasks SET priority=4 WHERE id = '3; DROP TABLE test;'"

In [39]:
cursor.execute(query, (id_number,))
conn.commit()

InvalidTextRepresentation: invalid input syntax for type integer: "3; DROP TABLE test;"
LINE 1: UPDATE tasks SET priority=4 WHERE id = '3; DROP TABLE test;'
                                               ^

### SELECT

In [41]:
select_query = "SELECT * FROM tasks"

In [44]:
cursor.execute(select_query)

results = cursor.fetchall()

In [45]:
results

[(1, 'opis zadania', 2, True),
 (2, 'zrobić obiad', 3, False),
 (3, 'opis zadania', 4, True)]

### DELETE

In [46]:
# delete_query = "DELETE FROM tasks WHERE id=1"
delete_query = "DELETE FROM tasks WHERE id=2 RETURNING *"

In [47]:
cursor.execute(delete_query)
record = cursor.fetchone()
conn.commit()

In [48]:
record

(2, 'zrobić obiad', 3, False)